In [40]:
import numpy as np

k = 5


def func_g(x, y):
    return np.sin(3 * x + 4 * y)


def generate_A(N):
    h = 1 / N
    size = (N - 1) ** 2
    const = h ** 2 * k ** 2
    A = np.zeros((size, size))
    b = np.zeros(size)
    for i in range(size):
        for j in range(i, size):
            if i == j:
                A[i, j] = A[j, i] = (24 - 4 * const) / 9
            elif (i == j - 1 and j % (N - 1) != 0)  or j - i == N - 1:
                A[i, j] = A[j, i] = (-3 - const) / 9
            elif (j - i == N and j % (N - 1) != 0) or (j - i == N - 2 and i % (N - 1) != 0):
                A[i, j] = A[j, i] = (-12 - const) / 36

        # b =
        # (12 + const) / 36 * func_g(0, 0) + (3 + const) / 9 * (func_g(h, 0) + func_g(0, h))
        #  (12 + const) / 36 * func_g(1, 0) + (3 + const) / 9 * (func_g(1-h, 0) + func_g(1, h))
        # (12 + const) / 36 * func_g(0, 1) + (3 + const) / 9 * (func_g(h, 1) + func_g(0, 1-h))
        # (12 + const) / 36 * func_g(1, 1) + (3 + const) / 9 * (func_g(1-h, 1) + func_g(1, 1-h))

    print(A)
    return A


A = generate_A(3)

[[ 1.43209877 -0.64197531 -0.64197531 -0.41049383]
 [-0.64197531  1.43209877 -0.41049383 -0.64197531]
 [-0.64197531 -0.41049383  1.43209877 -0.64197531]
 [-0.41049383 -0.64197531 -0.64197531  1.43209877]]


In [41]:
A_4 = np.array([
    [1.972222222222222, -0.5069444444444444, 0.0, -0.5069444444444444, -0.3767361111111111, 0.0, 0.0, 0.0, 0.0],
    [-0.5069444444444444, 1.972222222222222, -0.5069444444444444, -0.3767361111111111, -0.5069444444444444,
     -0.3767361111111111, 0.0, 0.0, 0.0],
    [0.0, -0.5069444444444444, 1.972222222222222, 0.0, -0.3767361111111111, -0.5069444444444444, 0.0, 0.0, 0.0],
    [-0.5069444444444444, -0.3767361111111111, 0.0, 1.972222222222222, -0.5069444444444444, 0.0, -0.5069444444444444,
     -0.3767361111111111, 0.0],
    [-0.3767361111111111, -0.5069444444444444, -0.3767361111111111, -0.5069444444444444, 1.972222222222222,
     -0.5069444444444444, -0.3767361111111111, -0.5069444444444444, -0.3767361111111111],
    [0.0, -0.3767361111111111, -0.5069444444444444, 0.0, -0.5069444444444444, 1.972222222222222, 0.0,
     -0.3767361111111111, -0.5069444444444444],
    [0.0, 0.0, 0.0, -0.5069444444444444, -0.3767361111111111, 0.0, 1.972222222222222, -0.5069444444444444, 0.0],
    [0.0, 0.0, 0.0, -0.3767361111111111, -0.5069444444444444, -0.3767361111111111, -0.5069444444444444,
     1.972222222222222, -0.5069444444444444],
    [0.0, 0.0, 0.0, 0.0, -0.3767361111111111, -0.5069444444444444, 0.0, -0.5069444444444444, 1.972222222222222],
])
A_3 = np.array([
    [1.4320987654320987, -0.6419753086419753, -0.6419753086419753, -0.4104938271604938],
    [-0.6419753086419753, 1.4320987654320987, -0.4104938271604938, -0.6419753086419753],
    [-0.6419753086419753, -0.4104938271604938, 1.4320987654320987, -0.6419753086419753],
    [-0.4104938271604938, -0.6419753086419753, -0.6419753086419753, 1.4320987654320987],
])

In [42]:
assert np.allclose(A,A_3)